In [2]:
!python -V

Python 3.9.19


In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb

from sklearn.metrics import mean_squared_error, root_mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import pickle

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("nyc-taxi-experiments")

<Experiment: artifact_location='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/mlruns/2', creation_time=1716478091983, experiment_id='2', last_update_time=1716478091983, lifecycle_stage='active', name='nyc-taxi-experiments', tags={}>

In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [8]:
len(df_train), len(df_val)

(73908, 61921)

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [10]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
with mlflow.start_run():

    mlflow.set_tag("develop", "R.Sanghvi")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("val-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.001
    lr = Lasso(alpha)
    mlflow.log_param("alpha", alpha)

    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)


In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:11.64459                          
[1]	validation-rmse:11.12765                          
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:32:03] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[2]	validation-rmse:10.65883                          
[3]	validation-rmse:10.23484                          
[4]	validation-rmse:9.85072                           
[5]	validation-rmse:9.50490                           
[6]	validation-rmse:9.19317                           
[7]	validation-rmse:8.91281                           
[8]	validation-rmse:8.66155                           
[9]	validation-rmse:8.43655                           
[10]	validation-rmse:8.23471                          
[11]	validation-rmse:8.05433                          
[12]	validation-rmse:7.89365                          
[13]	validation-rmse:7.75090                          
[14]	validation-rmse:7.62319                          
[15]	validation-rmse:7.50985                          
[16]	validation-rmse:7.40843                          
[17]	validation-rmse:7.31772                          
[18]	validation-rmse:7.23774                          
[19]	validation-rmse:7.16607                          
[20]	valid

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:32:26] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.06806                                                    
[1]	validation-rmse:6.60073                                                    
[2]	validation-rmse:6.53279                                                    
[3]	validation-rmse:6.51155                                                    
[4]	validation-rmse:6.50486                                                    
[5]	validation-rmse:6.49580                                                    
[6]	validation-rmse:6.49251                                                    
[7]	validation-rmse:6.48355                                                    
[8]	validation-rmse:6.47871                                                    
[9]	validation-rmse:6.46805                                                    
[10]	validation-rmse:6.46327                                                   
[11]	validation-rmse:6.45728                                                   
[12]	validation-rmse:6.45269            

...

[1]	validation-rmse:11.21592                                                    
[2]	validation-rmse:10.77939                                                    
[3]	validation-rmse:10.38039                                                    
[4]	validation-rmse:10.01605                                                    
[5]	validation-rmse:9.68556                                                     
[6]	validation-rmse:9.38328                                                     
[7]	validation-rmse:9.10930                                                     
[8]	validation-rmse:8.86038                                                     
[9]	validation-rmse:8.63494                                                     
[10]	validation-rmse:8.43179                                                    
[11]	validation-rmse:8.24844                                                    
[12]	validation-rmse:8.08150                                                    
[13]	validation-rmse:7.93152

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:49:02] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:6.70119                                                     
[4]	validation-rmse:6.69011                                                     
[5]	validation-rmse:6.68725                                                     
[6]	validation-rmse:6.68692                                                     
[7]	validation-rmse:6.68064                                                     
[8]	validation-rmse:6.67954                                                     
[9]	validation-rmse:6.67887                                                     
[10]	validation-rmse:6.67378                                                    
[11]	validation-rmse:6.66942                                                    
[12]	validation-rmse:6.66444                                                    
[13]	validation-rmse:6.65923                                                    
[14]	validation-rmse:6.66098                                                    
[15]	validation-rmse:6.65776

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:49:07] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.47037                                                    
[1]	validation-rmse:10.81894                                                    
[2]	validation-rmse:10.24637                                                    
[3]	validation-rmse:9.74742                                                     
[4]	validation-rmse:9.31375                                                     
[5]	validation-rmse:8.93798                                                     
[6]	validation-rmse:8.61216                                                     
[7]	validation-rmse:8.32464                                                     
[8]	validation-rmse:8.07929                                                     
[9]	validation-rmse:7.87057                                                     
[10]	validation-rmse:7.68507                                                    
[11]	validation-rmse:7.53407                                                    
[12]	validation-rmse:7.40129

In [22]:
# Training model with best hyperparameter
params = {
    "reg_lambda": 0.35931960489862685,
    "seed": 42,
    "max_depth": 37,
    "min_child_weight": 1.2139033434560909,
    "learning_rate": 0.11035059238513262,
    "reg_alpha": 0.05628231573565957,
    "objective": "reg:linear"
}

mlflow.xgboost.autolog()
booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)

2024/05/23 17:54:20 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bc62efa19aaa4815a2a2f31733dda747', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:11.32560


/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:54:21] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:10.56512
[2]	validation-rmse:9.91606
[3]	validation-rmse:9.36434
[4]	validation-rmse:8.89930
[5]	validation-rmse:8.50849
[6]	validation-rmse:8.18315
[7]	validation-rmse:7.91196
[8]	validation-rmse:7.68522
[9]	validation-rmse:7.49580
[10]	validation-rmse:7.33888
[11]	validation-rmse:7.20580
[12]	validation-rmse:7.09734
[13]	validation-rmse:7.00805
[14]	validation-rmse:6.93310
[15]	validation-rmse:6.86976
[16]	validation-rmse:6.81682
[17]	validation-rmse:6.77136
[18]	validation-rmse:6.73385
[19]	validation-rmse:6.70130
[20]	validation-rmse:6.67571
[21]	validation-rmse:6.65214
[22]	validation-rmse:6.63081
[23]	validation-rmse:6.61403
[24]	validation-rmse:6.59795
[25]	validation-rmse:6.58504
[26]	validation-rmse:6.57302
[27]	validation-rmse:6.56317
[28]	validation-rmse:6.55341
[29]	validation-rmse:6.54597
[30]	validation-rmse:6.53908
[31]	validation-rmse:6.53153
[32]	validation-rmse:6.52613
[33]	validation-rmse:6.52097
[34]	validation-rmse:6.51612
[35]	validation-rmse:6

2024/05/23 17:55:02 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/23 17:55:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:55:02] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


In [24]:
mlflow.xgboost.autolog(disable=True)
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    # Training model with best hyperparameter
    best_params = {
        "reg_lambda": 0.35931960489862685,
        "seed": 42,
        "max_depth": 37,
        "min_child_weight": 1.2139033434560909,
        "learning_rate": 0.11035059238513262,
        "reg_alpha": 0.05628231573565957,
        "objective": "reg:linear"
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
                params=best_params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(valid, 'validation')],
                early_stopping_rounds=50
            )
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:11.32560


/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	validation-rmse:10.56512
[2]	validation-rmse:9.91606
[3]	validation-rmse:9.36434
[4]	validation-rmse:8.89930
[5]	validation-rmse:8.50849
[6]	validation-rmse:8.18315
[7]	validation-rmse:7.91196
[8]	validation-rmse:7.68522
[9]	validation-rmse:7.49580
[10]	validation-rmse:7.33888
[11]	validation-rmse:7.20580
[12]	validation-rmse:7.09734
[13]	validation-rmse:7.00805
[14]	validation-rmse:6.93310
[15]	validation-rmse:6.86976
[16]	validation-rmse:6.81682
[17]	validation-rmse:6.77136
[18]	validation-rmse:6.73385
[19]	validation-rmse:6.70130
[20]	validation-rmse:6.67571
[21]	validation-rmse:6.65214
[22]	validation-rmse:6.63081
[23]	validation-rmse:6.61403
[24]	validation-rmse:6.59795
[25]	validation-rmse:6.58504
[26]	validation-rmse:6.57302
[27]	validation-rmse:6.56317
[28]	validation-rmse:6.55341
[29]	validation-rmse:6.54597
[30]	validation-rmse:6.53908
[31]	validation-rmse:6.53153
[32]	validation-rmse:6.52613
[33]	validation-rmse:6.52097
[34]	validation-rmse:6.51612
[35]	validation-rmse:6

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:57:33] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutil

In [25]:
logged_model = 'runs:/044fa9b574e04760b0462ae1e375fb02/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:58:40] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [26]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 044fa9b574e04760b0462ae1e375fb02

In [27]:
xgboost_model = mlflow.xgboost.load_model(logged_model)
xgboost_model

/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:58:46] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [28]:
y_pred = xgboost_model.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)
rmse

6.300452789009226

In [12]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/05/23 18:02:41 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/23 18:08:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
2024/05/23 18:08:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/opt/anaconda3/envs/mlops_env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 an